# data_to_analysis.py

- From Data to Matched Analysis

This code chunk contains to processes. First, we go from the datasets to a match with a certain election, to a match with the nearest winner. That dataset is then used to find the wealths of those politicians, and then we can compute an estimate of the average winner and average loser wealth. 

- From Data to Unconditional Analysis

Secondly, we start again from the datasets, and we go to margins. This dataset is merged with a dataset of politicians with the margins (conditional on the margin being <$x$%, but not necessarily from the same elections. 

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## From Data to Matched Analysis

First step: 

- Import the data

In [2]:
data1 = pd.read_csv("../Administration/new_data_entry_file.csv").iloc[:,1:]
data2 = pd.read_csv("../Administration/new_data_entry_file_lessclose.csv")

nonpoliticians = pd.concat([data1, data2])
nonpoliticians = nonpoliticians[nonpoliticians['Gevonden'] == 'ja']
nonpoliticians['jaar'] = nonpoliticians['Verkiezingdatum'].astype(str).str.split('/').apply(lambda x : x[2])

nonpoliticians.head(2)

,Naam,District,Verkiezingdatum,InTKEK,Gevonden,Gebdatum,Sterfdatum,Sterfplaats,Vermogen,Link1,Link2,Polpartij,Opm,jaar
0,mr. J. van Riemsdijk,Almelo,02/01/1849,nee,ja,NaN,01/11/1879,Almelo,29618.27,https://www.genealogieonline.nl/en/genealogie-...,NaN,NaN,NaN,1849
7,D. Mulder Dzn.,Winschoten,12/03/1878,nee,ja,27/11/1825,17/09/1889,Amsterdam,1018.44,https://genealogiewinschoten.nl/diversen/oud-n...,https://noord-hollandsarchief.nl/bronnen/archi...,NaN,NaN,1878


Second step:

- Import the `allelected.csv` dataset
- Then, left merge the nonpoliticians data with the `allected.csv` data 
    - This choice: we leave 'double' matches in the dataset: one nonpolitician can have two or more politician matches
    - Detailed approach: first, match on exact election date
    - If that doesn't work, match on election year

In [3]:
def decode_accents(name):
    try:
        return name.encode('latin-1').decode('raw_unicode_escape').encode('latin-1').decode('utf-8')
    except:
        return name

In [4]:
#allelected = pd.read_csv("../Data/elections/allelected.csv", encoding='latin1')#
all_candidates_elections = pd.read_csv("../Data/elections/election_results_details.csv", encoding='utf-8-sig')
all_candidates_elections["Naam"] = all_candidates_elections["Naam"].apply(lambda x : decode_accents(x))
#print(all_candidates_elections.shape)
all_candidates_elections.head(2)

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel
0,0,mr. B. Wichers,NaN,700,83.73%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418
1,1,W.L. de Sturler,NaN,34,4.07%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418


In [5]:
allelected = pd.read_csv("../Data/elections/allelected.csv", encoding='latin1')
print(allelected.shape)
allelected.head(2)

(2508, 12)


,achternaam,voornaam,tussenvoegsel,jaar,maand,dag,type verkiezing,districtsnaam,aantal stemmen,omvang_electoraat,zetels,drempel
0,Aalberse,P.J.M.,NaN,1903,2,18,tussentijds,Almelo,3821,7865,1.0,2953.0
1,Aalberse,P.J.M.,NaN,1905,6,16,algemeen,Almelo,5217,9324,1.0,3922.0


In [6]:
# Clean the data a little bit

allelected['maand'] = np.where(allelected['maand'] < 10, '0' + allelected['maand'].astype(str), allelected['maand'])
allelected['dag'] = np.where(allelected['dag'] < 10, '0' + allelected['dag'].astype(str), allelected['dag'])
allelected["Verkiezingdatum"] = allelected.dag.astype(str).str.cat(others=[allelected.maand.astype(str), allelected.jaar.astype(str)], sep='/')
allelected.rename(columns={'districtsnaam':'District'}, inplace = True)

allelected.head(2)

,achternaam,voornaam,tussenvoegsel,jaar,maand,dag,type verkiezing,District,aantal stemmen,omvang_electoraat,zetels,drempel,Verkiezingdatum
0,Aalberse,P.J.M.,NaN,1903,02,18,tussentijds,Almelo,3821,7865,1.0,2953.0,18/02/1903
1,Aalberse,P.J.M.,NaN,1905,06,16,algemeen,Almelo,5217,9324,1.0,3922.0,16/06/1905


In [7]:
all_candidates_elections["jaar"] = all_candidates_elections["Verkiezingdatum"].apply(lambda x : int(x.split("/")[2]))
all_candidates_elections["maand"] = all_candidates_elections["Verkiezingdatum"].apply(lambda x : int(x.split("/")[1]))
all_candidates_elections["dag"] = all_candidates_elections["Verkiezingdatum"].apply(lambda x : int(x.split("/")[0]))
all_candidates_elections.head(2)

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,jaar,maand,dag
0,0,mr. B. Wichers,NaN,700,83.73%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418,1848,11,30
1,1,W.L. de Sturler,NaN,34,4.07%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418,1848,11,30


In [8]:
# Extract the number of seats for each elections from the allelected df
hoeveelzetels = allelected.groupby(['District','Verkiezingdatum'])['zetels'].mean()
hoeveelzetels.head(2)

hoeveelzetels["Alkmaar"]["01/03/1892"]

1.0

In [9]:
def to_int(x):
    try:
        return int(x)
    except:
        return None
    
all_candidates_elections['Aantal stemmen'] = all_candidates_elections['Aantal stemmen'].apply(to_int)

In [12]:
grouped_allelec = all_candidates_elections.groupby(['District', 'Verkiezingdatum']).apply(lambda x : x.sort_values(['Aantal stemmen'], ascending = False))
grouped_allelec = grouped_allelec.reset_index(level=-1, drop=True)
grouped_allelec.head(2)

Unnamed: 0                 Naam Aanbevolen door  \
District Verkiezingdatum                                                    
Alkmaar  01/03/1892             4934  mr. W. van der Kaay             NaN   
         01/09/1887             2126  mr. W. van der Kaay             NaN   

                          Aantal stemmen Procentueel District Verkiezingdatum  \
District Verkiezingdatum                                                        
Alkmaar  01/03/1892               1283.0      97.05%  Alkmaar      01/03/1892   
         01/09/1887               1356.0      75.42%  Alkmaar      01/09/1887   

                                 Type  Omvang electoraat Opkomst  \
District Verkiezingdatum                                           
Alkmaar  01/03/1892       tussentijds               4783    1399   
         01/09/1887          algemeen               4165    1835   

                         Aantal stembriefjes Aantal stemmen geldig  \
District Verkiezingdatum                                             
Alkmaar  01/03/1892                     1398                  1322   
         01/09/1887                     1837                  3596   

                         Aantal stemmen blanco Aantal zetels Kiesdrempel  \
District Verkiezingdatum                                                   
Alkmaar  01/03/1892                         74             1         661   
         01/09/1887                         72             2         899   

                          jaar  maand  dag  
District Verkiezingdatum                    
Alkmaar  01/03/1892       1892      3    1  
         01/09/1887       1887      9    1

In [13]:
a = grouped_allelec.loc[("Alkmaar", "15/06/1917")]
a

,,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,jaar,maand,dag
District,Verkiezingdatum,,,,,,,,,,,,,,,,,,
Alkmaar,15/06/1917,8114,jhr.mr. P. van Foreest,VL,NaN,enige kandidaat,Alkmaar,15/06/1917,algemeen/enkelvoudig,12557,-,-,-,-,-,-,1917,6,15


In [15]:
# Brief outline togo chto ja xochu

allwinners = []

for i in all_candidates_elections.groupby(['District', 'Verkiezingdatum']).groups:
    
    cur_election = grouped_allelec.loc[(i[0], i[1])]
    
    # Ischi skolko sideniy v dataframe hoeveelzetels i pishi eto v peremenuiu
    try:
        skolkosidenij = hoeveelzetels[i[0]][i[1]]
    except:
        skolkosidenij = cur_election['Aantal zetels'].iloc[0]
    # Vybrat stolko nabliudenij skolko ukazano v tolkto chto cdelannoi peremennoy
    try:
        int(skolkosidenij)
    except:
        continue
    cur_winners = cur_election.head(int(skolkosidenij))
    allwinners.append(cur_winners)

allwinners_df = pd.concat(allwinners).reset_index(drop=True)
allwinners_df.head(5)

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,jaar,maand,dag
0,4934,mr. W. van der Kaay,NaN,1283.0,97.05%,Alkmaar,01/03/1892,tussentijds,4783,1399,1398,1322,74,1,661,1892,3,1
1,2126,mr. W. van der Kaay,NaN,1356.0,75.42%,Alkmaar,01/09/1887,algemeen,4165,1835,1837,3596,72,2,899,1887,9,1
2,2127,mr. J.L. de Bruyn Kops,NaN,1348.0,74.97%,Alkmaar,01/09/1887,algemeen,4165,1835,1837,3596,72,2,899,1887,9,1
3,2079,jhr.mr. C. van Foreest,NaN,1444.0,57.85%,Alkmaar,04/02/1868,herstemming,3232,2507,2509,2496,8,1,1248,1868,2,4
4,4940,A.P. de Lange,NaN,1961.0,63.28%,Alkmaar,05/06/1894,tussentijds,4779,3124,3124,3099,21,1,1550,1894,6,5


In [16]:
# What to do?

## Step 1: Take all nonpoliticians
## Merge it with allwinners_df on the basis of District, Verkiezingsdatum
nonpoliticians.merge(allwinners_df, how="left", left_on=['District', 'Verkiezingdatum'], right_on = ['District', 'Verkiezingdatum'])

## Join with how='left', left nonpoliticians, 
## Check what to do in case of double matches (I want to keep all)

,Naam_x,District,Verkiezingdatum,InTKEK,Gevonden,Gebdatum,Sterfdatum,Sterfplaats,Vermogen,Link1,...,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,jaar_y,maand,dag
0,mr. J. van Riemsdijk,Almelo,02/01/1849,nee,ja,NaN,01/11/1879,Almelo,29618.27,https://www.genealogieonline.nl/en/genealogie-...,...,547,480,481,480,1,1,240,1849,1,2
1,D. Mulder Dzn.,Winschoten,12/03/1878,nee,ja,27/11/1825,17/09/1889,Amsterdam,1018.44,https://genealogiewinschoten.nl/diversen/oud-n...,...,2558,1609,1609,1594,8,1,797,1878,3,12
2,D. Mulder Dzn.,Winschoten,10/06/1879,nee,ja,27/11/1825,17/09/1889,Amsterdam,1018.44,https://genealogiewinschoten.nl/diversen/oud-n...,...,2628,1229,1229,1219,5,1,610,1879,6,10
3,mr. S.J. baron van Pallandt,Almelo,08/06/1875,nee,ja,NaN,NaN,NaN,1076713.3,NaN,...,2281,2019,2019,2006,6,1,1003,1875,6,8
4,mr. S.J. baron van Pallandt,Almelo,22/06/1875,nee,ja,NaN,NaN,NaN,1076713.3,NaN,...,2281,2110,2110,2098,11,1,1049,1875,6,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,mr. W.R. baron van Tuyll van Serooskerke,Utrecht,30/10/1866,nee,ja,NaN,09/04/1888,Den Haag,560405,https://schiedam.courant.nu/issue/NSC/1888-04-...,...,2944,2212,2212,4362,48,2,1091,1866,10,30
233,J.D. Kruseman,Hoorn,20/04/1852,nee,ja,NaN,13/01/1861,Den Haag,NaN,https://www.genealogieonline.nl/stamboom-de-ka...,...,3430,1661,1661,1642,9,1,821,1852,4,20
234,S. de Vries,Hoorn,20/04/1852,nee,ja,02/02/1825,13/11/1916,Hoorn,NaN,https://gw.geneanet.org/herediasittig?lang=en&...,...,3430,1661,1661,1642,9,1,821,1852,4,20
235,H. Koomen,Hoorn,30/07/1858,nee,ja,NaN,16/05/1884,Winkel (Noord-Holland),NaN,https://wit-west-friesland.nl/getperson.php?pe...,...,3679,1409,1409,1396,4,1,698,1858,7,30


In [10]:
election_to_winners = {}
for i in tqdm(range(len(all_candidates_elections))):
    row = all_candidates_elections.iloc[i]
    district = row["District"]
    year = row["jaar"]
    month = row["maand"]
    day = row["dag"]
    if (district, year, month, day) in election_to_winners:
        continue
    district_winners = allelected[allelected["districtsnaam"] == district]
    year_winners = district_winners[district_winners["jaar"] == year]
    month_winners = year_winners[year_winners["maand"] == month]
    winners = month_winners[month_winners["dag"] == day]
    election_to_winners[(district, year, month, day)] = winners

100%|██████████| 8238/8238 [00:10<00:00, 785.09it/s] 


In [327]:
name_in_winners = []
for i in tqdm(range(len(all_candidates_elections))):
    row = all_candidates_elections.iloc[i]
    district = row["District"]
    year = row["jaar"]
    month = row["maand"]
    day = row["dag"]
    winners = election_to_winners[(district, year, month, day)]
    name = row["Naam"]
    winner_names = winners["naam"].values
    if any(name in n for n in winner_names) or any(n in name for n in winner_names):
        name_in_winners.append(True)
    else:
        name_in_winners.append(False)
all_candidates_elections = all_candidates_elections.assign(name_in_winners = name_in_winners)
all_candidates_elections.head(2)

100%|██████████| 8238/8238 [00:01<00:00, 4279.70it/s]


,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,jaar,maand,dag,name_in_winners
0,0,mr. B. Wichers,NaN,700,83.73%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418,1848,11,30,True
1,1,W.L. de Sturler,NaN,34,4.07%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418,1848,11,30,False


In [330]:
matched_candidate_names = set(all_candidates_elections[all_candidates_elections["name_in_winners"]]["Naam"])
missed_winners = []
for i, row in tqdm(allelected.iterrows()):
    winner_name = row["naam"]
    if not any(winner_name in n for n in matched_candidate_names) and not any(n in winner_name for n in matched_candidate_names):
        missed_winners.append(i)
missed_winners_df = allelected[allelected.index.isin(missed_winners)]
missed_winners_df

2508it [00:00, 6139.40it/s]


,achternaam,voornaam,tussenvoegsel,jaar,maand,dag,type verkiezing,districtsnaam,aantal stemmen,omvang_electoraat,zetels,drempel,naam
407,Bylandt van Mariënweerd,O.W.A. graaf van,NaN,1849,7,10,herstemming,Zaltbommel,431,935,1.0,307.0,O.W.A. graaf van Bylandt van Mariënweerd
439,Costerus,P.J.,NaN,1848,12,28,herstemming,Sneek,354,1168,1.0,343.0,P.J. Costerus
1062,Hoytema,W.J. van,NaN,1849,8,6,tussentijds,Zaltbommel,279,935,1.0,267.0,W.J. van Hoytema
1439,Lycklama à Nijeholt,W.H.,NaN,1858,6,8,periodiek,Sneek,700,2842,1.0,662.0,W.H. Lycklama à Nijeholt
1440,Lycklama à Nijeholt,W.H.,NaN,1862,6,24,herstemming,Sneek,1025,2859,1.0,953.0,W.H. Lycklama à Nijeholt
2187,Thomassen à Thuessink van der Hoop,A.J.,NaN,1879,6,24,herstemming,Steenwijk,711,1539,1.0,678.0,A.J. Thomassen à Thuessink van der Hoop
2188,Thomassen à Thuessink van der Hoop,G.H.,NaN,1882,4,11,tussentijds,Steenwijk,718,1591,1.0,672.0,G.H. Thomassen à Thuessink van der Hoop
2189,Thomassen à Thuessink van der Hoop,G.H.,NaN,1883,6,12,periodiek,Steenwijk,772,1646,1.0,717.0,G.H. Thomassen à Thuessink van der Hoop
2190,Thomassen à Thuessink van der Hoop,G.H.,NaN,1884,10,28,algemeen,Steenwijk,719,1631,1.0,650.0,G.H. Thomassen à Thuessink van der Hoop
2294,Villers de Pité,L.L.G.M. de,NaN,1849,1,29,tussentijds,Heerlen,200,759,1.0,194.0,L.L.G.M. de Villers de Pité


Then, the task is to find the politician `b1_nummer` in the `politician_names` dataset, and subsequently, wealth on the `wealth.csv` dataset